In [4]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, row_number, rank, dense_rank, lag, lead, sum, avg
from pyspark.sql.window import Window

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
data = [
    (1, "Alice",   "HR",       3500, "2022-01-10"),
    (2, "Bob",     "HR",       4200, "2022-01-15"),
    (3, "Charlie", "IT",       6000, "2022-02-01"),
    (4, "David",   "IT",       7200, "2022-02-10"),
    (5, "Eve",     "Finance",  5100, "2022-03-05"),
    (6, "Frank",   "Finance",  6200, "2022-03-20"),
    (7, "Grace",   "IT",       5500, "2022-04-01"),
    (8, "Hank",    "HR",       3000, "2022-04-12"),
    (9, "Ivy",     "Finance",  7000, "2022-05-02"),
    (10,"Jack",    "IT",       8000, "2022-05-15")
]
columns = ["emp_id", "name", "dept", "salary", "hire_date"]
df = spark.createDataFrame(data, columns)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
df.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+-------+-------+------+----------+
|emp_id|   name|   dept|salary| hire_date|
+------+-------+-------+------+----------+
|     1|  Alice|     HR|  3500|2022-01-10|
|     2|    Bob|     HR|  4200|2022-01-15|
|     3|Charlie|     IT|  6000|2022-02-01|
|     4|  David|     IT|  7200|2022-02-10|
|     5|    Eve|Finance|  5100|2022-03-05|
|     6|  Frank|Finance|  6200|2022-03-20|
|     7|  Grace|     IT|  5500|2022-04-01|
|     8|   Hank|     HR|  3000|2022-04-12|
|     9|    Ivy|Finance|  7000|2022-05-02|
|    10|   Jack|     IT|  8000|2022-05-15|
+------+-------+-------+------+----------+

#### row_number, rank, dense_rank

In [5]:
windowSpec = Window.partitionBy("dept").orderBy(col("salary").desc())

df.withColumn("row_number", row_number().over(windowSpec)) \
  .withColumn("rank", rank().over(windowSpec)) \
  .withColumn("dense_rank", dense_rank().over(windowSpec)) \
  .show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+-------+-------+------+----------+----------+----+----------+
|emp_id|   name|   dept|salary| hire_date|row_number|rank|dense_rank|
+------+-------+-------+------+----------+----------+----+----------+
|     9|    Ivy|Finance|  7000|2022-05-02|         1|   1|         1|
|     6|  Frank|Finance|  6200|2022-03-20|         2|   2|         2|
|     5|    Eve|Finance|  5100|2022-03-05|         3|   3|         3|
|     2|    Bob|     HR|  4200|2022-01-15|         1|   1|         1|
|     1|  Alice|     HR|  3500|2022-01-10|         2|   2|         2|
|     8|   Hank|     HR|  3000|2022-04-12|         3|   3|         3|
|    10|   Jack|     IT|  8000|2022-05-15|         1|   1|         1|
|     4|  David|     IT|  7200|2022-02-10|         2|   2|         2|
|     3|Charlie|     IT|  6000|2022-02-01|         3|   3|         3|
|     7|  Grace|     IT|  5500|2022-04-01|         4|   4|         4|
+------+-------+-------+------+----------+----------+----+----------+

In [7]:
windowSpec = Window.partitionBy("dept").orderBy(col("salary").asc())

df.withColumn("row_number", row_number().over(windowSpec)) \
  .withColumn("rank", rank().over(windowSpec)) \
  .withColumn("dense_rank", dense_rank().over(windowSpec)) \
  .show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+-------+-------+------+----------+----------+----+----------+
|emp_id|   name|   dept|salary| hire_date|row_number|rank|dense_rank|
+------+-------+-------+------+----------+----------+----+----------+
|     5|    Eve|Finance|  5100|2022-03-05|         1|   1|         1|
|     6|  Frank|Finance|  6200|2022-03-20|         2|   2|         2|
|     9|    Ivy|Finance|  7000|2022-05-02|         3|   3|         3|
|     8|   Hank|     HR|  3000|2022-04-12|         1|   1|         1|
|     1|  Alice|     HR|  3500|2022-01-10|         2|   2|         2|
|     2|    Bob|     HR|  4200|2022-01-15|         3|   3|         3|
|     7|  Grace|     IT|  5500|2022-04-01|         1|   1|         1|
|     3|Charlie|     IT|  6000|2022-02-01|         2|   2|         2|
|     4|  David|     IT|  7200|2022-02-10|         3|   3|         3|
|    10|   Jack|     IT|  8000|2022-05-15|         4|   4|         4|
+------+-------+-------+------+----------+----------+----+----------+

#### Lead & Lag

In [8]:
df.withColumn("next_salary", lead("salary", 1).over(windowSpec)) \
  .withColumn("prev_salary", lag("salary", 1).over(windowSpec)) \
  .show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+-------+-------+------+----------+-----------+-----------+
|emp_id|   name|   dept|salary| hire_date|next_salary|prev_salary|
+------+-------+-------+------+----------+-----------+-----------+
|     5|    Eve|Finance|  5100|2022-03-05|       6200|       null|
|     6|  Frank|Finance|  6200|2022-03-20|       7000|       5100|
|     9|    Ivy|Finance|  7000|2022-05-02|       null|       6200|
|     8|   Hank|     HR|  3000|2022-04-12|       3500|       null|
|     1|  Alice|     HR|  3500|2022-01-10|       4200|       3000|
|     2|    Bob|     HR|  4200|2022-01-15|       null|       3500|
|     7|  Grace|     IT|  5500|2022-04-01|       6000|       null|
|     3|Charlie|     IT|  6000|2022-02-01|       7200|       5500|
|     4|  David|     IT|  7200|2022-02-10|       8000|       6000|
|    10|   Jack|     IT|  8000|2022-05-15|       null|       7200|
+------+-------+-------+------+----------+-----------+-----------+

#### Aggregates with Window

In [9]:
aggSpec = Window.partitionBy("dept")

df.withColumn("dept_total", _sum("salary").over(aggSpec)) \
  .withColumn("dept_avg", avg("salary").over(aggSpec)) \
  .withColumn("running_total", _sum("salary").over(windowSpec.rowsBetween(Window.unboundedPreceding, 0))) \
  .show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+-------+-------+------+----------+----------+------------------+-------------+
|emp_id|   name|   dept|salary| hire_date|dept_total|          dept_avg|running_total|
+------+-------+-------+------+----------+----------+------------------+-------------+
|     5|    Eve|Finance|  5100|2022-03-05|     18300|            6100.0|         5100|
|     6|  Frank|Finance|  6200|2022-03-20|     18300|            6100.0|        11300|
|     9|    Ivy|Finance|  7000|2022-05-02|     18300|            6100.0|        18300|
|     8|   Hank|     HR|  3000|2022-04-12|     10700|3566.6666666666665|         3000|
|     1|  Alice|     HR|  3500|2022-01-10|     10700|3566.6666666666665|         6500|
|     2|    Bob|     HR|  4200|2022-01-15|     10700|3566.6666666666665|        10700|
|     7|  Grace|     IT|  5500|2022-04-01|     26700|            6675.0|         5500|
|     3|Charlie|     IT|  6000|2022-02-01|     26700|            6675.0|        11500|
|     4|  David|     IT|  7200|2022-02-10| 

### New DataFrame

In [14]:
data = [
    (1, "Alice", "HR", "New York", 3500, "2022-01-10"),
    (2, "Bob", "HR", "New York", 4200, "2022-01-15"),
    (3, "Charlie", "IT", "Chicago", 6000, "2022-02-01"),
    (4, "David", "IT", "Chicago", 7200, "2022-02-10"),
    (5, "Eve", "Finance", "New York", 5100, "2022-03-05"),
    (6, "Frank", "Finance", "Boston", 6200, "2022-03-20"),
    (7, "Grace", "IT", "Boston", 5500, "2022-04-01"),
    (8, "Hank", "HR", "Boston", 3000, "2022-04-12"),
    (9, "Ivy", "Finance", "Chicago", 7000, "2022-05-02"),
    (10,"Jack", "IT", "Boston", 8000, "2022-05-15"),
    (11,"Ken", "Finance", "New York", 4800, "2022-06-01"),
    (12,"Lily", "HR", "Chicago", 3900, "2022-06-10")
]

columns = ["emp_id", "name", "dept", "location", "salary", "hire_date"]

df = spark.createDataFrame(data, columns)
df.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+-------+-------+--------+------+----------+
|emp_id|   name|   dept|location|salary| hire_date|
+------+-------+-------+--------+------+----------+
|     1|  Alice|     HR|New York|  3500|2022-01-10|
|     2|    Bob|     HR|New York|  4200|2022-01-15|
|     3|Charlie|     IT| Chicago|  6000|2022-02-01|
|     4|  David|     IT| Chicago|  7200|2022-02-10|
|     5|    Eve|Finance|New York|  5100|2022-03-05|
|     6|  Frank|Finance|  Boston|  6200|2022-03-20|
|     7|  Grace|     IT|  Boston|  5500|2022-04-01|
|     8|   Hank|     HR|  Boston|  3000|2022-04-12|
|     9|    Ivy|Finance| Chicago|  7000|2022-05-02|
|    10|   Jack|     IT|  Boston|  8000|2022-05-15|
|    11|    Ken|Finance|New York|  4800|2022-06-01|
|    12|   Lily|     HR| Chicago|  3900|2022-06-10|
+------+-------+-------+--------+------+----------+

In [15]:
win = Window.partitionBy("location", "dept").orderBy(col("salary").asc())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
df.withColumn("row_number", row_number().over(win)) \
    .withColumn("rank", rank().over(win)) \
    .withColumn("dense_rank", dense_rank().over(win)).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+-------+-------+--------+------+----------+----------+----+----------+
|emp_id|   name|   dept|location|salary| hire_date|row_number|rank|dense_rank|
+------+-------+-------+--------+------+----------+----------+----+----------+
|     6|  Frank|Finance|  Boston|  6200|2022-03-20|         1|   1|         1|
|     8|   Hank|     HR|  Boston|  3000|2022-04-12|         1|   1|         1|
|     7|  Grace|     IT|  Boston|  5500|2022-04-01|         1|   1|         1|
|    10|   Jack|     IT|  Boston|  8000|2022-05-15|         2|   2|         2|
|     9|    Ivy|Finance| Chicago|  7000|2022-05-02|         1|   1|         1|
|    12|   Lily|     HR| Chicago|  3900|2022-06-10|         1|   1|         1|
|     3|Charlie|     IT| Chicago|  6000|2022-02-01|         1|   1|         1|
|     4|  David|     IT| Chicago|  7200|2022-02-10|         2|   2|         2|
|    11|    Ken|Finance|New York|  4800|2022-06-01|         1|   1|         1|
|     5|    Eve|Finance|New York|  5100|2022-03-05| 

In [19]:
df.withColumn("lead", lead("salary", 1).over(Window.partitionBy("dept").orderBy(col("salary").asc()))).withColumn("lag", lag("salary", 1).over(win)).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+-------+-------+--------+------+----------+----+----+
|emp_id|   name|   dept|location|salary| hire_date|lead| lag|
+------+-------+-------+--------+------+----------+----+----+
|     6|  Frank|Finance|  Boston|  6200|2022-03-20|7000|null|
|     8|   Hank|     HR|  Boston|  3000|2022-04-12|3500|null|
|     7|  Grace|     IT|  Boston|  5500|2022-04-01|6000|null|
|    10|   Jack|     IT|  Boston|  8000|2022-05-15|null|5500|
|     9|    Ivy|Finance| Chicago|  7000|2022-05-02|null|null|
|    12|   Lily|     HR| Chicago|  3900|2022-06-10|4200|null|
|     3|Charlie|     IT| Chicago|  6000|2022-02-01|7200|null|
|     4|  David|     IT| Chicago|  7200|2022-02-10|8000|6000|
|    11|    Ken|Finance|New York|  4800|2022-06-01|5100|null|
|     5|    Eve|Finance|New York|  5100|2022-03-05|6200|4800|
|     1|  Alice|     HR|New York|  3500|2022-01-10|3900|null|
|     2|    Bob|     HR|New York|  4200|2022-01-15|null|3500|
+------+-------+-------+--------+------+----------+----+----+

#### rowsBetween

In [21]:
df.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+-------+-------+--------+------+----------+
|emp_id|   name|   dept|location|salary| hire_date|
+------+-------+-------+--------+------+----------+
|     1|  Alice|     HR|New York|  3500|2022-01-10|
|     2|    Bob|     HR|New York|  4200|2022-01-15|
|     3|Charlie|     IT| Chicago|  6000|2022-02-01|
|     4|  David|     IT| Chicago|  7200|2022-02-10|
|     5|    Eve|Finance|New York|  5100|2022-03-05|
|     6|  Frank|Finance|  Boston|  6200|2022-03-20|
|     7|  Grace|     IT|  Boston|  5500|2022-04-01|
|     8|   Hank|     HR|  Boston|  3000|2022-04-12|
|     9|    Ivy|Finance| Chicago|  7000|2022-05-02|
|    10|   Jack|     IT|  Boston|  8000|2022-05-15|
|    11|    Ken|Finance|New York|  4800|2022-06-01|
|    12|   Lily|     HR| Chicago|  3900|2022-06-10|
+------+-------+-------+--------+------+----------+

In [22]:
win = Window.partitionBy("dept").orderBy("salary")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
df.withColumn("Running Salary Total", sum("salary").over(win.rowsBetween(Window.unboundedPreceding, 0))).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+-------+-------+--------+------+----------+--------------------+
|emp_id|   name|   dept|location|salary| hire_date|Running Salary Total|
+------+-------+-------+--------+------+----------+--------------------+
|    11|    Ken|Finance|New York|  4800|2022-06-01|                4800|
|     5|    Eve|Finance|New York|  5100|2022-03-05|                9900|
|     6|  Frank|Finance|  Boston|  6200|2022-03-20|               16100|
|     9|    Ivy|Finance| Chicago|  7000|2022-05-02|               23100|
|     8|   Hank|     HR|  Boston|  3000|2022-04-12|                3000|
|     1|  Alice|     HR|New York|  3500|2022-01-10|                6500|
|    12|   Lily|     HR| Chicago|  3900|2022-06-10|               10400|
|     2|    Bob|     HR|New York|  4200|2022-01-15|               14600|
|     7|  Grace|     IT|  Boston|  5500|2022-04-01|                5500|
|     3|Charlie|     IT| Chicago|  6000|2022-02-01|               11500|
|     4|  David|     IT| Chicago|  7200|2022-02-10|

In [24]:
df.withColumn("Running Salary Total", sum("salary").over(win.rowsBetween(-1, 1))).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+-------+-------+--------+------+----------+--------------------+
|emp_id|   name|   dept|location|salary| hire_date|Running Salary Total|
+------+-------+-------+--------+------+----------+--------------------+
|    11|    Ken|Finance|New York|  4800|2022-06-01|                9900|
|     5|    Eve|Finance|New York|  5100|2022-03-05|               16100|
|     6|  Frank|Finance|  Boston|  6200|2022-03-20|               18300|
|     9|    Ivy|Finance| Chicago|  7000|2022-05-02|               13200|
|     8|   Hank|     HR|  Boston|  3000|2022-04-12|                6500|
|     1|  Alice|     HR|New York|  3500|2022-01-10|               10400|
|    12|   Lily|     HR| Chicago|  3900|2022-06-10|               11600|
|     2|    Bob|     HR|New York|  4200|2022-01-15|                8100|
|     7|  Grace|     IT|  Boston|  5500|2022-04-01|               11500|
|     3|Charlie|     IT| Chicago|  6000|2022-02-01|               18700|
|     4|  David|     IT| Chicago|  7200|2022-02-10|

In [25]:
df.withColumn("Running Salary Total", sum("salary").over(win.rowsBetween(-1, Window.unboundedFollowing))).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+-------+-------+--------+------+----------+--------------------+
|emp_id|   name|   dept|location|salary| hire_date|Running Salary Total|
+------+-------+-------+--------+------+----------+--------------------+
|    11|    Ken|Finance|New York|  4800|2022-06-01|               23100|
|     5|    Eve|Finance|New York|  5100|2022-03-05|               23100|
|     6|  Frank|Finance|  Boston|  6200|2022-03-20|               18300|
|     9|    Ivy|Finance| Chicago|  7000|2022-05-02|               13200|
|     8|   Hank|     HR|  Boston|  3000|2022-04-12|               14600|
|     1|  Alice|     HR|New York|  3500|2022-01-10|               14600|
|    12|   Lily|     HR| Chicago|  3900|2022-06-10|               11600|
|     2|    Bob|     HR|New York|  4200|2022-01-15|                8100|
|     7|  Grace|     IT|  Boston|  5500|2022-04-01|               26700|
|     3|Charlie|     IT| Chicago|  6000|2022-02-01|               26700|
|     4|  David|     IT| Chicago|  7200|2022-02-10|

#### rangeBetween

In [26]:
df.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+-------+-------+--------+------+----------+
|emp_id|   name|   dept|location|salary| hire_date|
+------+-------+-------+--------+------+----------+
|     1|  Alice|     HR|New York|  3500|2022-01-10|
|     2|    Bob|     HR|New York|  4200|2022-01-15|
|     3|Charlie|     IT| Chicago|  6000|2022-02-01|
|     4|  David|     IT| Chicago|  7200|2022-02-10|
|     5|    Eve|Finance|New York|  5100|2022-03-05|
|     6|  Frank|Finance|  Boston|  6200|2022-03-20|
|     7|  Grace|     IT|  Boston|  5500|2022-04-01|
|     8|   Hank|     HR|  Boston|  3000|2022-04-12|
|     9|    Ivy|Finance| Chicago|  7000|2022-05-02|
|    10|   Jack|     IT|  Boston|  8000|2022-05-15|
|    11|    Ken|Finance|New York|  4800|2022-06-01|
|    12|   Lily|     HR| Chicago|  3900|2022-06-10|
+------+-------+-------+--------+------+----------+

In [27]:
win = Window.partitionBy("dept").orderBy(col("salary").desc())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [30]:
df.withColumn("Running Total Salary", sum("salary").over(win.rangeBetween(-100, 100))).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+-------+-------+--------+------+----------+--------------------+
|emp_id|   name|   dept|location|salary| hire_date|Running Total Salary|
+------+-------+-------+--------+------+----------+--------------------+
|     9|    Ivy|Finance| Chicago|  7000|2022-05-02|                7000|
|     6|  Frank|Finance|  Boston|  6200|2022-03-20|                6200|
|     5|    Eve|Finance|New York|  5100|2022-03-05|                5100|
|    11|    Ken|Finance|New York|  4800|2022-06-01|                4800|
|     2|    Bob|     HR|New York|  4200|2022-01-15|                4200|
|    12|   Lily|     HR| Chicago|  3900|2022-06-10|                3900|
|     1|  Alice|     HR|New York|  3500|2022-01-10|                3500|
|     8|   Hank|     HR|  Boston|  3000|2022-04-12|                3000|
|    10|   Jack|     IT|  Boston|  8000|2022-05-15|                8000|
|     4|  David|     IT| Chicago|  7200|2022-02-10|                7200|
|     3|Charlie|     IT| Chicago|  6000|2022-02-01|